# Spotify - Tunnel Playlist
## What makes a good tunnel playlist?
The goal of this project is to looking into [Spotify's Tunnel playlist](https://open.spotify.com/playlist/0OwZcV6bAN2jqqXmA23Jnw?si=b01b353689a34be8) and identify the key track elements that makes for a good tunnel track.

In [21]:
import config
import pandas as pd
import json
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth

## Get Authentication

In [2]:
redirect_uri = 'http://localhost:7777/callback'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=config.client_id,
                                               client_secret=config.client_secret,
                                               redirect_uri=redirect_uri,
                                               scope="user-read-recently-played"))

In [36]:
a_json['tracks']['items'][i]['track']['popularity']

9

In [49]:
track_id_ls = []
track_name_ls = []
track_pop_ls = []
artist_id_ls = []
artist_name_ls = []

playlist_id = '0OwZcV6bAN2jqqXmA23Jnw?si=7e0fd30e5bd44bfa'

results = sp.playlist(playlist_id, fields=None, market=None, additional_types=('track', ))

json1 = json.dumps(results, indent=4)
a_json = json.loads(json1)
print(len(a_json['tracks']['items']))


for i in range(len(a_json['tracks']['items'])):
    track_id_ls.append(a_json['tracks']['items'][i]['track']['id'])
    track_name_ls.append(a_json['tracks']['items'][i]['track']['name'])
    track_pop_ls.append(a_json['tracks']['items'][i]['track']['popularity'])
    artist_id_ls.append(a_json['tracks']['items'][i]['track']['album']['artists'][0]['id'])
    artist_name_ls.append(a_json['tracks']['items'][i]['track']['album']['artists'][0]['name'])

44


In [53]:
playlist_df = pd.DataFrame(list(zip(track_id_ls, track_name_ls, track_pop_ls, artist_id_ls, artist_name_ls)),
               columns =['track_id', 'track_name', 'track_popularity','artist_id', 'artist_name'])
playlist_df

,track_id,track_name,track_popularity,artist_id,artist_name
0,0I1eFRytp4XRhLCjT6tZm7,I Can't Handle Change,85,7dIpKWlEeAljA20vFJ82RD,Roar
1,0ntQJM78wzOLVeCUAW7Y45,Sex on Fire,84,2qk9voo8llSGYcZ6xrBzKx,Kings of Leon
2,6N22FZs2ZhPBYi3b9XPajV,Still Don't Know My Name,82,2feDdbD5araYcm6JhFHHw7,Labrinth
3,1Fwj0wThn3kTg8D7KgWdsU,My Tears Are Becoming A Sea,64,63MQldklfxkjYDoUE4Tppz,M83
4,7Jh1bpe76CNTCgdgAdBw4Z,Heroes - 2017 Remaster,77,0oSGxfWSnnOXhD2fKuz2Gy,David Bowie
5,45S5WTQEGOB1VHr1Q4FuPl,Golden,86,6KImCVD70vtIoJWnq6nGn3,Harry Styles
6,1eyzqe2QqGZUmfcPZtrIyt,Midnight City,75,63MQldklfxkjYDoUE4Tppz,M83
7,3GCdLUSnKSMJhs4Tj6CV3s,All The Stars (with SZA),83,2YZyLoL8N0Wb9xBt1NhZWg,Kendrick Lamar
8,6VzcQuzTNTMFnJ6rBSaLH9,Fine Line,81,6KImCVD70vtIoJWnq6nGn3,Harry Styles
9,5SO2xwVkn1iu8f5eSxvSdR,Symphonia IX,50,0m5FakHKCQdA7UN0PIzMcL,Current Joys


## Get Audio Features for All Tracks

In [66]:
results2 = sp.audio_features(tracks=track_id_ls)
track_df = pd.json_normalize(results2)
track_df.rename(columns={'id':'track_id'}, inplace=True)
track_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,track_id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.247,0.438,5,-8.478,1,0.0358,0.044700,0.000867,0.1110,0.3900,185.711,audio_features,0I1eFRytp4XRhLCjT6tZm7,spotify:track:0I1eFRytp4XRhLCjT6tZm7,https://api.spotify.com/v1/tracks/0I1eFRytp4XR...,https://api.spotify.com/v1/audio-analysis/0I1e...,198213,3
1,0.542,0.905,9,-5.653,1,0.0540,0.001720,0.010400,0.1360,0.3740,153.398,audio_features,0ntQJM78wzOLVeCUAW7Y45,spotify:track:0ntQJM78wzOLVeCUAW7Y45,https://api.spotify.com/v1/tracks/0ntQJM78wzOL...,https://api.spotify.com/v1/audio-analysis/0ntQ...,203347,4
2,0.316,0.630,5,-5.963,1,0.1170,0.471000,0.262000,0.2050,0.3130,89.643,audio_features,6N22FZs2ZhPBYi3b9XPajV,spotify:track:6N22FZs2ZhPBYi3b9XPajV,https://api.spotify.com/v1/tracks/6N22FZs2ZhPB...,https://api.spotify.com/v1/audio-analysis/6N22...,153294,1
3,0.194,0.376,4,-11.356,1,0.0463,0.110000,0.386000,0.5930,0.0723,48.637,audio_features,1Fwj0wThn3kTg8D7KgWdsU,spotify:track:1Fwj0wThn3kTg8D7KgWdsU,https://api.spotify.com/v1/tracks/1Fwj0wThn3kT...,https://api.spotify.com/v1/audio-analysis/1Fwj...,151827,3
4,0.490,0.758,7,-6.491,1,0.0297,0.000671,0.484000,0.0920,0.4350,112.114,audio_features,7Jh1bpe76CNTCgdgAdBw4Z,spotify:track:7Jh1bpe76CNTCgdgAdBw4Z,https://api.spotify.com/v1/tracks/7Jh1bpe76CNT...,https://api.spotify.com/v1/audio-analysis/7Jh1...,371413,4
5,0.448,0.838,4,-5.257,0,0.0557,0.210000,0.000131,0.1310,0.2540,139.863,audio_features,45S5WTQEGOB1VHr1Q4FuPl,spotify:track:45S5WTQEGOB1VHr1Q4FuPl,https://api.spotify.com/v1/tracks/45S5WTQEGOB1...,https://api.spotify.com/v1/audio-analysis/45S5...,208907,4
6,0.526,0.712,11,-6.525,0,0.0356,0.016100,0.000000,0.1790,0.3200,105.009,audio_features,1eyzqe2QqGZUmfcPZtrIyt,spotify:track:1eyzqe2QqGZUmfcPZtrIyt,https://api.spotify.com/v1/tracks/1eyzqe2QqGZU...,https://api.spotify.com/v1/audio-analysis/1eyz...,241440,4
7,0.698,0.633,8,-4.946,1,0.0597,0.060500,0.000194,0.0926,0.5520,96.924,audio_features,3GCdLUSnKSMJhs4Tj6CV3s,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,https://api.spotify.com/v1/tracks/3GCdLUSnKSMJ...,https://api.spotify.com/v1/audio-analysis/3GCd...,232187,4
8,0.306,0.347,2,-8.500,1,0.0334,0.172000,0.000130,0.0485,0.0511,120.996,audio_features,6VzcQuzTNTMFnJ6rBSaLH9,spotify:track:6VzcQuzTNTMFnJ6rBSaLH9,https://api.spotify.com/v1/tracks/6VzcQuzTNTMF...,https://api.spotify.com/v1/audio-analysis/6Vzc...,377960,4
9,0.593,0.380,0,-12.682,1,0.0247,0.913000,0.573000,0.2890,0.1490,99.323,audio_features,5SO2xwVkn1iu8f5eSxvSdR,spotify:track:5SO2xwVkn1iu8f5eSxvSdR,https://api.spotify.com/v1/tracks/5SO2xwVkn1iu...,https://api.spotify.com/v1/audio-analysis/5SO2...,181360,4


### Bring track popularity into track_df

In [70]:
track_df_main = track_df.merge(playlist_df, on='track_id', how='left')
track_df_main

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,track_id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,track_popularity,artist_id,artist_name
0,0.247,0.438,5,-8.478,1,0.0358,0.044700,0.000867,0.1110,0.3900,...,0I1eFRytp4XRhLCjT6tZm7,spotify:track:0I1eFRytp4XRhLCjT6tZm7,https://api.spotify.com/v1/tracks/0I1eFRytp4XR...,https://api.spotify.com/v1/audio-analysis/0I1e...,198213,3,I Can't Handle Change,85,7dIpKWlEeAljA20vFJ82RD,Roar
1,0.542,0.905,9,-5.653,1,0.0540,0.001720,0.010400,0.1360,0.3740,...,0ntQJM78wzOLVeCUAW7Y45,spotify:track:0ntQJM78wzOLVeCUAW7Y45,https://api.spotify.com/v1/tracks/0ntQJM78wzOL...,https://api.spotify.com/v1/audio-analysis/0ntQ...,203347,4,Sex on Fire,84,2qk9voo8llSGYcZ6xrBzKx,Kings of Leon
2,0.316,0.630,5,-5.963,1,0.1170,0.471000,0.262000,0.2050,0.3130,...,6N22FZs2ZhPBYi3b9XPajV,spotify:track:6N22FZs2ZhPBYi3b9XPajV,https://api.spotify.com/v1/tracks/6N22FZs2ZhPB...,https://api.spotify.com/v1/audio-analysis/6N22...,153294,1,Still Don't Know My Name,82,2feDdbD5araYcm6JhFHHw7,Labrinth
3,0.194,0.376,4,-11.356,1,0.0463,0.110000,0.386000,0.5930,0.0723,...,1Fwj0wThn3kTg8D7KgWdsU,spotify:track:1Fwj0wThn3kTg8D7KgWdsU,https://api.spotify.com/v1/tracks/1Fwj0wThn3kT...,https://api.spotify.com/v1/audio-analysis/1Fwj...,151827,3,My Tears Are Becoming A Sea,64,63MQldklfxkjYDoUE4Tppz,M83
4,0.490,0.758,7,-6.491,1,0.0297,0.000671,0.484000,0.0920,0.4350,...,7Jh1bpe76CNTCgdgAdBw4Z,spotify:track:7Jh1bpe76CNTCgdgAdBw4Z,https://api.spotify.com/v1/tracks/7Jh1bpe76CNT...,https://api.spotify.com/v1/audio-analysis/7Jh1...,371413,4,Heroes - 2017 Remaster,77,0oSGxfWSnnOXhD2fKuz2Gy,David Bowie
5,0.448,0.838,4,-5.257,0,0.0557,0.210000,0.000131,0.1310,0.2540,...,45S5WTQEGOB1VHr1Q4FuPl,spotify:track:45S5WTQEGOB1VHr1Q4FuPl,https://api.spotify.com/v1/tracks/45S5WTQEGOB1...,https://api.spotify.com/v1/audio-analysis/45S5...,208907,4,Golden,86,6KImCVD70vtIoJWnq6nGn3,Harry Styles
6,0.526,0.712,11,-6.525,0,0.0356,0.016100,0.000000,0.1790,0.3200,...,1eyzqe2QqGZUmfcPZtrIyt,spotify:track:1eyzqe2QqGZUmfcPZtrIyt,https://api.spotify.com/v1/tracks/1eyzqe2QqGZU...,https://api.spotify.com/v1/audio-analysis/1eyz...,241440,4,Midnight City,75,63MQldklfxkjYDoUE4Tppz,M83
7,0.698,0.633,8,-4.946,1,0.0597,0.060500,0.000194,0.0926,0.5520,...,3GCdLUSnKSMJhs4Tj6CV3s,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,https://api.spotify.com/v1/tracks/3GCdLUSnKSMJ...,https://api.spotify.com/v1/audio-analysis/3GCd...,232187,4,All The Stars (with SZA),83,2YZyLoL8N0Wb9xBt1NhZWg,Kendrick Lamar
8,0.306,0.347,2,-8.500,1,0.0334,0.172000,0.000130,0.0485,0.0511,...,6VzcQuzTNTMFnJ6rBSaLH9,spotify:track:6VzcQuzTNTMFnJ6rBSaLH9,https://api.spotify.com/v1/tracks/6VzcQuzTNTMF...,https://api.spotify.com/v1/audio-analysis/6Vzc...,377960,4,Fine Line,81,6KImCVD70vtIoJWnq6nGn3,Harry Styles
9,0.593,0.380,0,-12.682,1,0.0247,0.913000,0.573000,0.2890,0.1490,...,5SO2xwVkn1iu8f5eSxvSdR,spotify:track:5SO2xwVkn1iu8f5eSxvSdR,https://api.spotify.com/v1/tracks/5SO2xwVkn1iu...,https://api.spotify.com/v1/audio-analysis/5SO2...,181360,4,Symphonia IX,50,0m5FakHKCQdA7UN0PIzMcL,Current Joys


## Sort by Popularity of Track

In [71]:
sorted_df = track_df_main.sort_values('track_popularity', ascending = False)
sorted_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,track_id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,track_popularity,artist_id,artist_name
5,0.448,0.838,4,-5.257,0,0.0557,0.21000,0.000131,0.131,0.254,...,45S5WTQEGOB1VHr1Q4FuPl,spotify:track:45S5WTQEGOB1VHr1Q4FuPl,https://api.spotify.com/v1/tracks/45S5WTQEGOB1...,https://api.spotify.com/v1/audio-analysis/45S5...,208907,4,Golden,86,6KImCVD70vtIoJWnq6nGn3,Harry Styles
20,0.611,0.797,6,-7.627,0,0.0533,0.00543,0.001370,0.260,0.518,...,2GiJYvgVaD2HtM8GqD9EgQ,spotify:track:2GiJYvgVaD2HtM8GqD9EgQ,https://api.spotify.com/v1/tracks/2GiJYvgVaD2H...,https://api.spotify.com/v1/audio-analysis/2GiJ...,218107,4,Electric Love,86,1KP6TWI40m7p3QBTU6u2xo,BØRNS
0,0.247,0.438,5,-8.478,1,0.0358,0.04470,0.000867,0.111,0.390,...,0I1eFRytp4XRhLCjT6tZm7,spotify:track:0I1eFRytp4XRhLCjT6tZm7,https://api.spotify.com/v1/tracks/0I1eFRytp4XR...,https://api.spotify.com/v1/audio-analysis/0I1e...,198213,3,I Can't Handle Change,85,7dIpKWlEeAljA20vFJ82RD,Roar
25,0.645,0.795,7,-12.095,1,0.0527,0.34700,0.003890,0.104,0.535,...,4RvWPyQ5RL0ao9LPZeSouE,spotify:track:4RvWPyQ5RL0ao9LPZeSouE,https://api.spotify.com/v1/tracks/4RvWPyQ5RL0a...,https://api.spotify.com/v1/audio-analysis/4RvW...,251489,4,Everybody Wants To Rule The World,85,4bthk9UfsYUYdcFyqxmSUU,Tears For Fears
37,0.574,0.664,11,-6.068,1,0.0409,0.04480,0.055300,0.146,0.226,...,6EtKlIQmGPB9SX8UjDJG5s,spotify:track:6EtKlIQmGPB9SX8UjDJG5s,https://api.spotify.com/v1/tracks/6EtKlIQmGPB9...,https://api.spotify.com/v1/audio-analysis/6EtK...,91870,4,Formula,84,2feDdbD5araYcm6JhFHHw7,Labrinth


## Analyise the Data